<a href="https://colab.research.google.com/github/GergelyTuri/sleep/blob/master/notebooks/Significant_cells.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook for calculating cells which significatly upregulated during sleep
* 11/27/2023 - setup
* 2/13/2024 - modified version after refactoring

In [ ]:
!git clone https://github.com/GergelyTuri/sleep.git

fatal: destination path 'sleep' already exists and is not an empty directory.


In [ ]:
# %cd /content/sleep/
# !git pull

In [ ]:
%cd /content/sleep/
!pip install .

In [ ]:
import sys
from os.path import join
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
# sys.path.append('/content/sleep/notebooks/neuropil_analysis/')
# import neuropil as npil
sys.path.append("/content/sleep/src/")
from eeg_class import eegData
from behavior_class import behaviorData
import custom_statistics

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
data_path = '/gdrive/Shareddrives/Turi_lab/Data/Sleep/2p/Analysis/data/5HT2afl05b_1/7_12_SD2'
file_name = 'dfof_mc.csv'

## Loading dfof data

In [ ]:
dfof_data = pd.read_csv(join(data_path, file_name))
dfof_data.set_index('roi_label', drop=True, inplace=True)
dfof_data.head()

## Loading sleep and other behavior data (velocity)

In [ ]:
sleep_data = pd.read_csv(join(data_path, 'velo_eeg.csv'))
sleep_data.head()

### Determining awake mobile and imobile periods

In [ ]:
sleep_data['mobile_immobile'] = behaviorData.define_immobility(
        sleep_data["filtered velo"])
sleep_data.head()

Plotting some of it to make sure it worked

In [ ]:
sleep_data["filtered velo"][:100].plot()
sleep_data['mobile_immobile'][:100].astype(int).plot()

### Creating brain state filters for filtering the datasets

In [ ]:
brain_states =  eegData.brain_state_filter(
        velo_eeg_df=sleep_data,
        states=["awake_mobile", "awake_immobile", "NREM", "REM", "other"],
    )

Creating more filters...

In [ ]:
nrem_mask = brain_states['NREM']
awake_mask = brain_states['awake_immobile']

In [ ]:
nrem_intervals = dfof_data.loc[:, nrem_mask.values]
awake_intervals = dfof_data.loc[:, awake_mask.values]

### Calculating significant cells

In [ ]:
significant_cells_nrem_vs_awake = custom_statistics.significance_calc(nrem_intervals.to_numpy(),
                                                      awake_intervals.to_numpy())

In [ ]:
significant_cells_nrem_vs_awake

array([13, 32, 34, 39])

In [ ]:
significant_cells_awake_vs_nrem = custom_statistics.significance_calc(awake_intervals.to_numpy(),
                                                      nrem_intervals.to_numpy())

In [ ]:
significant_cells_awake_vs_nrem

### filtering out significant and non significant cells from the original data

In [ ]:
sig_cell_mask = np.zeros(dfof_data.shape[0], dtype=bool)
sig_cell_mask[significant_cells_nrem_vs_awake] = True
significant_cell_data = dfof_data[sig_cell_mask]
non_significant_cell_data = dfof_data.iloc[~sig_cell_mask]

In [ ]:
print(f"significant cells: {significant_cell_data.index} \n Non significant cells: {non_significant_cell_data.index}")